# **# 🧱 Cell–0: Colab Setup (TF, utils)**

In [ ]:
# ===== Cell 0: Base setup =====
!pip install -q tensorflow==2.15.0 scikit-learn matplotlib flask flask-cors pyngrok -U

import os, json, random, warnings, glob, io, time
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from collections import Counter

SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

print(tf.__version__)


ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.0
2.19.0


# **🗂️ Cell–১: Drive  + paths**

In [ ]:
# ===== Cell 1: Drive & paths =====
from google.colab import drive
drive.mount('/content/drive')

# EDIT THESE to your paths
BUSI_DIR = "/content/drive/MyDrive/CSE-51/8Th/Capstone/Alternative data/Breast Ultra Image/Dataset_BUSI_with_GT"
OUT_DIR  = "/content/drive/MyDrive/CSE-51/8Th/Capstone/Breast_Models"

os.makedirs(OUT_DIR, exist_ok=True)

CLASSES = ["benign", "malignant", "normal"]  # fixed order (stable API)
class_to_idx = {c:i for i,c in enumerate(CLASSES)}
idx_to_class = {i:c for c,i in class_to_idx.items()}

print("BUSI_DIR:", BUSI_DIR)
print("OUT_DIR :", OUT_DIR)
print("Classes :", CLASSES)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BUSI_DIR: /content/drive/MyDrive/CSE-51/8Th/Capstone/Alternative data/Breast Ultra Image/Dataset_BUSI_with_GT
OUT_DIR : /content/drive/MyDrive/CSE-51/8Th/Capstone/Breast_Models
Classes : ['benign', 'malignant', 'normal']



# **📥 Cell 2: Data load gather & split + tf.data**




In [ ]:
# ===== Cell 2: Gather & split =====
paths, labels = [], []
for cls in CLASSES:
    cdir = os.path.join(BUSI_DIR, cls)
    imgs = sorted(glob.glob(os.path.join(cdir, "*.png"))) + \
           sorted(glob.glob(os.path.join(cdir, "*.jpg"))) + \
           sorted(glob.glob(os.path.join(cdir, "*.jpeg")))
    imgs = [p for p in imgs if not p.lower().endswith("_mask.png")]
    paths.extend(imgs)
    labels.extend([cls]*len(imgs))

assert len(paths) > 0, "No images found. Check BUSI_DIR."
y = np.array([class_to_idx[l] for l in labels])

for c in CLASSES:
    print(f"{c}: {sum(1 for L in labels if L==c)}")

with open(os.path.join(OUT_DIR, "labels.json"), "w") as f:
    json.dump({str(k):v for k,v in idx_to_class.items()}, f, indent=2)

X_temp, X_test, y_temp, y_test = train_test_split(paths, y, test_size=0.15, stratify=y, random_state=SEED)
X_train, X_val,  y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, stratify=y_temp, random_state=SEED)
print(f"Split → train={len(X_train)} | val={len(X_val)} | test={len(X_test)}")

IMG_SIZE   = (224, 224)
BATCH_SIZE = 32

@tf.function
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="augment")

def build_ds(X, y, training):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    ds = ds.shuffle(len(X), seed=SEED) if training else ds
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.map(lambda a,b: (aug(a, training=True), b), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

ds_train = build_ds(X_train, y_train, training=True)
ds_val   = build_ds(X_val,   y_val,   training=False)
ds_test  = build_ds(X_test,  y_test,  training=False)

cnt = Counter(y_train)
total = sum(cnt.values())
class_weight = {i: total/(len(CLASSES)*cnt.get(i,1)) for i in range(len(CLASSES))}
print("Class weights:", class_weight)


benign: 454
malignant: 211
normal: 133
Split → train=558 | val=120 | test=120
Class weights: {0: 0.5849056603773585, 1: 1.2653061224489797, 2: 2.0}


# **🧠** Cell 3 : Model Traning

In [ ]:
# ===== Cell 3: Train & fine-tune =====
base = tf.keras.applications.MobileNetV2(include_top=False, weights="imagenet",
                                         input_shape=IMG_SIZE+(3,))
base.trainable = False

inputs = keras.Input(shape=IMG_SIZE+(3,))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(CLASSES), activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

ckpt = os.path.join(OUT_DIR, "busi_mobilenetv2_best.keras")
callbacks = [
    keras.callbacks.ModelCheckpoint(ckpt, monitor="val_accuracy", save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=6, restore_best_weights=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
]

print("Training (frozen)…")
model.fit(ds_train, validation_data=ds_val, epochs=20,
          class_weight=class_weight, callbacks=callbacks, verbose=1)

base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

print("Fine-tuning (top layers)…")
model.fit(ds_train, validation_data=ds_val, epochs=10,
          class_weight=class_weight, callbacks=callbacks, verbose=1)

best_model = keras.models.load_model(ckpt)

# Evaluate
test_loss, test_acc = best_model.evaluate(ds_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f} | loss: {test_loss:.3f}")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training (frozen)…
Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.4275 - loss: 1.4247
Epoch 1: val_accuracy improved from -inf to 0.52500, saving model to /content/drive/MyDrive/CSE-51/8Th/Capstone/Breast_Models/busi_mobilenetv2_best.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 266s 13s/step - accuracy: 0.4261 - loss: 1.4200 - val_accuracy: 0.5250 - val_loss: 0.9914 - learning_rate: 0.0010
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 532ms/step - accuracy: 0.5032 - loss: 1.0357
Epoch 2: val_accuracy improved from 0.52500 to 0.55833, saving model to /content/drive/MyDrive/CSE-51/8Th/Capstone/Breast_Models/busi_mobilenetv2_best.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 615ms/step - accuracy: 0.5041 - loss: 1.0345 - val_accuracy: 0.5583 - val_loss: 0.9552 - learning_rate: 0.0010
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 549ms/step - accuracy: 0.5654 - loss: 0.8641
Epoch 3: val_accuracy improved from 0.55833 to 0.61667, saving model to /content/d